XML Schema Creation

In [1]:
import MaterialX as mx

stdlib = mx.createDocument()
libFiles = []
searchPath = mx.getDefaultDataSearchPath()
libFiles = mx.loadLibraries(mx.getDefaultDataLibraryFolders(), searchPath, stdlib)


In [2]:
# Inspect the class hierarchy
# pulling out static attributes
# Note that Element is missing static attributes !
# Use getattr() to get the actual value of the attribute
import inspect
tab = ' '
indent = 1
for base_class in inspect.getmro(mx.Node):
    # Test if class is in MaterialX modules
    if 'MaterialX' in base_class.__module__:
        print(indent * ' ', 'Class: ' + base_class.__name__)
        for key, value in base_class.__dict__.items():
            if not key.startswith('__'):
                # Test if value is not a function
                if not callable(value):
                    print(indent * ' ', '- Enumerant: ' + key + ', Attribute String: \"' + getattr(base_class, key) + '\"')
        indent += 1

  Class: Node
  - Enumerant: CATEGORY, Attribute String: "node"
   Class: InterfaceElement
   - Enumerant: NODE_DEF_ATTRIBUTE, Attribute String: "nodedef"
    Class: TypedElement
    - Enumerant: TYPE_ATTRIBUTE, Attribute String: "type"
     Class: Element


In [3]:
#from xmlschema import XMLSchema, Element
from xml.dom import minidom

In [4]:
#
def addTopLevelList(parent):
    # Create complexType
    complexType = root.createElement('xs:complexType')
    parent.appendChild(complexType)
    # Create sequence
    sequence = root.createElement('xs:sequence')
    complexType.appendChild(sequence)
    return sequence

def addDefaultElementAttrib(parent):
    elemAttrib = ["name"]
    for a in elemAttrib:
        mtlxAttribute = root.createElement('xs:attribute')
        mtlxAttribute.setAttribute('name', a)
        mtlxAttribute.setAttribute('type', 'xs:string')
        mtlxAttribute.setAttribute('use', 'required' if (a == 'name') else 'optional')
        parent.appendChild(mtlxAttribute)

def addBaseElement(root, parent, mxType):

    # Get parent class if any
    extends = ''
    derived = inspect.getmro(mxType)
    if derived and len(derived) > 1:
        if 'MaterialX' in derived[1].__module__:
            #print(derived[0].__name__, 'extends: ', derived[1].__name__)
            extends = derived[1].__name__.lower()

    comment = root.createComment(' MaterialX Type Definitios: ' + mxType.__name__ + ' ')
    parent.appendChild(comment)    

    complexType = root.createElement('xs:complexType')
    complexType.setAttribute('name', mxType.__name__.lower())
    parent.appendChild(complexType)

    #sequence = root.createElement('xs:sequence')
    attrParent = None
    if extends:
        # Add extension parent
        complextContent = root.createElement('xs:complexContent')
        complexType.appendChild(complextContent)

        extension = root.createElement('xs:extension')
        extension.setAttribute('base', extends)
        complextContent.appendChild(extension)

        attrParent = extension             
    else:
        attrParent = complexType

    # Workaround for Element class which does not
    # declare any attributes statics    
    if mxType == mx.Element:
        names = [ 'name', 'fileprefix', 'geomprefix', 'colorspace', 'inherit', 'namespace', 'doc' ]

        for name in names: 
            mtlxAttribute = root.createElement('xs:attribute')
            mtlxAttribute.setAttribute('name', name)
            mtlxAttribute.setAttribute('type', 'xs:string')
            attrParent.appendChild(mtlxAttribute)    

    else:
        for key, value in mxType.__dict__.items():
            if not key.startswith('__') and key != 'CATEGORY':
                if not callable(value):                        
                    #print('Check key: ', getattr(mxType, key))
    
                    # Add attribute                        
                    mtlxAttribute = root.createElement('xs:attribute')
                    mtlxAttribute.setAttribute('name', getattr(mxType, key))
                    mtlxAttribute.setAttribute('type', 'xs:string')
                    attrParent.appendChild(mtlxAttribute)    

# Add Element type
def addMaterialxElement(root, parent, mxType):
    comment = root.createComment(' MaterialX Element Type: ' + mxType.__name__ + ' ')
    parent.appendChild(comment)    
    mtlxElement = root.createElement('xs:element')
    # Get lowercase name    
    mtlxElement.setAttribute('name', mxType.__name__.lower())
    parent.appendChild(mtlxElement)

    complexType = root.createElement('xs:complexType')
    mtlxElement.appendChild(complexType)

    for base_class in inspect.getmro(mxType):
        if 'MaterialX' in base_class.__module__:

            if (base_class == mx.Element):
                comment = root.createComment(' Inherted From: Element ')
                mtlxElement.appendChild(comment)    
                addDefaultElementAttrib(complexType)

            for key, value in base_class.__dict__.items():
                if not key.startswith('__') and key != 'CATEGORY':
                    if not callable(value):                        

                        if (mxType != base_class):
                            comment = root.createComment(' Inherted From: ' + base_class.__name__ + ' ')
                            complexType.appendChild(comment)    

                        # Add attribute                        
                        mtlxAttribute = root.createElement('xs:attribute')
                        mtlxAttribute.setAttribute('name', getattr(base_class, key))
                        mtlxAttribute.setAttribute('type', 'xs:string')
                        complexType.appendChild(mtlxAttribute)

root = minidom.Document()
xml = root.createElement('xs:schema')
xml.setAttribute('xmlns:xs', 'http://www.w3.org/2001/XMLSchema')
xml.setAttribute('attributeFormDefault', 'unqualified')
xml.setAttribute('elementFormDefault', 'qualified')
#xml.setAttribute('targetNamespace', 'http://www.materialx.org/')
xml.setAttribute('xmlns:mx', 'http://www.materialx.org/')

all_members = inspect.getmembers(mx, inspect.isclass)
classes = [member[1] for member in all_members if inspect.isclass(member[1])]

#classes = [member for member in all_members 
#            if inspect.isclass(member[1])]
for c in classes:
    if issubclass(c, getattr(mx, 'Element', object)):
        addBaseElement(root, xml, c)
#addBaseElement(root, xml, mx.InterfaceElement)
#addBaseElement(root, xml, mx.TypedElement)
#addBaseElement(root, xml, mx.TypeDef)
#addBaseElement(root, xml, mx.Unit)
#addBaseElement(root, xml, mx.UnitDef)
#addBaseElement(root, xml, mx.GeomPropDef)

mtlx = root.createElement('xs:element')
mtlx.setAttribute('name', 'materialx')
xml.appendChild(mtlx)

seq = addTopLevelList(mtlx)
#defs = root.createElement('xs:complexType')
#mtlx.appendChild(defs)

#seq = root.createElement('xs:sequence')
#defs.appendChild(seq)

maxOccurs = root.createElement('xs:choice')
maxOccurs.setAttribute('maxOccurs', 'unbounded')
seq.appendChild(maxOccurs)

defTypes = []
#[mx.UnitDef, mx.Unit, mx.TypeDef] 
#, mx.Node, mx.NodeGraph, mx.GeomPropDef, mx.Document, mx.NodeDef,
#            mx.Input, mx.Output]
for defType in defTypes:
    addMaterialxElement(root, seq, defType)

root.appendChild(xml)

data = root.toprettyxml(indent='  ')
print(data)

with open('./data/materialx_schema.xsd', 'w') as f:
    f.write(data)

<?xml version="1.0" ?>
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema" attributeFormDefault="unqualified" elementFormDefault="qualified" xmlns:mx="http://www.materialx.org/">
  <!-- MaterialX Type Definitios: AttributeDef -->
  <xs:complexType name="attributedef">
    <xs:complexContent>
      <xs:extension base="typedelement"/>
    </xs:complexContent>
  </xs:complexType>
  <!-- MaterialX Type Definitios: Backdrop -->
  <xs:complexType name="backdrop">
    <xs:complexContent>
      <xs:extension base="element">
        <xs:attribute name="contains" type="xs:string"/>
        <xs:attribute name="width" type="xs:string"/>
        <xs:attribute name="height" type="xs:string"/>
      </xs:extension>
    </xs:complexContent>
  </xs:complexType>
  <!-- MaterialX Type Definitios: Collection -->
  <xs:complexType name="collection">
    <xs:complexContent>
      <xs:extension base="element"/>
    </xs:complexContent>
  </xs:complexType>
  <!-- MaterialX Type Definitios: CommentElement -

In [5]:
#all_members = inspect.getmembers(mx, inspect.isclass)
#classes = [member for member in all_members if inspect.isclass(member[1])]

# Print the class names
#for class_name, _ in classes:
#    print(class_name)